How many items (and what items) is Country X the \#1 producter in for the most recent five years of data (2015-2020)? How many items is Country X the best producer (most efficient) producer in? 

In [1]:
import pandas as pd
import os

In [2]:
%run Methods.ipynb
%run TestProductionChange.ipynb

test_prepData (__main__.TestDataPrep) ... ok
test_withNA (__main__.TestDataPrep) ... ok
test_getItemsThatHaveElement (__main__.TestGetItemWithElements) ... ok
test_yieldPrepData (__main__.TestYieldDataPrep) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.131s

OK
test_prepData (__main__.TestDataPrep) ... ok
test_withNA (__main__.TestDataPrep) ... ok
test_getItemsThatHaveElement (__main__.TestGetItemWithElements) ... ok
test_yieldPrepData (__main__.TestYieldDataPrep) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.128s

OK
test_prepData (__main__.TestDataPrep) ... ok
test_withNA (__main__.TestDataPrep) ... ok
test_getItemsThatHaveElement (__main__.TestGetItemWithElements) ... ok
test_productionChange (__main__.TestItemToProdElement) ... ok
test_yieldPrepData (__main__.TestYieldDataPrep) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.176s

OK


In [3]:
pathToData = os.path.join(os.getcwd(), "Data", "Production_Crops_Livestock_E_All_Data.csv")
agData = pd.read_csv(pathToData, encoding="latin-1")
agData.head(2)

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2016,Y2016F,Y2017,Y2017F,Y2018,Y2018F,Y2019,Y2019F,Y2020,Y2020F
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,ha,NaN,NaN,NaN,...,19481.0,NaN,19793.0,NaN,20053.0,NaN,29203.0,NaN,22134.0,NaN
1,2,Afghanistan,221,"Almonds, with shell",5419,Yield,hg/ha,NaN,NaN,NaN,...,16859.0,Fc,13788.0,Fc,17161.0,Fc,13083.0,Fc,17759.0,Fc


# Clean the data

Drop the rows that are rollups - regions and China

In [4]:
agDataNoRegions = dropRegionRows(agData)
agDataClean = agDataNoRegions.drop(agDataNoRegions[agDataNoRegions['Area'] == 'China'].index)

Only want to see data for most recent 5 years 

In [5]:
fiveYearCols = createYearList(2015, 2020)
agDataFiveYears = agDataClean[["Area", "Item", "Element", "Unit"] + fiveYearCols]
agDataFiveYears.head(3)

,Area,Item,Element,Unit,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
0,Afghanistan,"Almonds, with shell",Area harvested,ha,14676.0,19481.0,19793.0,20053.0,29203.0,22134.0
1,Afghanistan,"Almonds, with shell",Yield,hg/ha,16521.0,16859.0,13788.0,17161.0,13083.0,17759.0
2,Afghanistan,"Almonds, with shell",Production,tonnes,24246.0,32843.0,27291.0,34413.0,38205.0,39307.0


For each item, get the it's relevant production element, then subset the data to only be production data.

In [6]:
agDataProduction = subsetProductionData(agDataFiveYears)
agDataProduction.head(3)

,Area,Item,Element,Unit,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020
2,Afghanistan,"Almonds, with shell",Production,tonnes,24246.0,32843.0,27291.0,34413.0,38205.0,39307.0
5,Afghanistan,"Anise, badian, fennel, coriander",Production,tonnes,18000.0,17333.0,18944.0,18093.0,18123.0,18387.0
8,Afghanistan,Apples,Production,tonnes,89733.0,140903.0,170443.0,217192.0,250324.0,270857.0


For NAN values, fill with 0. 

In [7]:
agDataProduction.fillna(0, inplace=True)

C:\Users\14159\AppData\Local\Temp\ipykernel_56408\677681761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agDataProduction.fillna(0, inplace=True)


Calculate totals across columns for each row

In [8]:
agDataProduction['Total'] = agDataProduction.loc[:, fiveYearCols].sum(axis = 1)
agDataProduction.head()

C:\Users\14159\AppData\Local\Temp\ipykernel_56408\638781829.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agDataProduction['Total'] = agDataProduction.loc[:, fiveYearCols].sum(axis = 1)


,Area,Item,Element,Unit,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Total
2,Afghanistan,"Almonds, with shell",Production,tonnes,24246.0,32843.0,27291.0,34413.0,38205.0,39307.0,196305.0
5,Afghanistan,"Anise, badian, fennel, coriander",Production,tonnes,18000.0,17333.0,18944.0,18093.0,18123.0,18387.0,108880.0
8,Afghanistan,Apples,Production,tonnes,89733.0,140903.0,170443.0,217192.0,250324.0,270857.0,1139452.0
11,Afghanistan,Apricots,Production,tonnes,87686.0,17894.0,131816.0,109086.0,129363.0,131788.0,607633.0
12,Afghanistan,Asses,Stocks,Head,1481000.0,1472100.0,1317000.0,1361141.0,1555247.0,1535435.0,8721923.0


Get a dataframe of the number one producers for each item

In [9]:
agDataN1 = agDataProduction.sort_values(['Total'], ascending = False).groupby('Item').head(1)

In [10]:
agDataN1['Area']

6796                 Brazil
6882                 Brazil
11206       China, mainland
11246       China, mainland
10812       China, mainland
                ...        
32380            Madagascar
44543    Russian Federation
34563                Mexico
23420                Guyana
49387                 Spain
Name: Area, Length: 300, dtype: object

Map country to items it is the number one producer in

In [11]:
countryCount = {}

for countryItemTuple in zip(agDataN1['Area'], agDataN1['Item']):
    curList = countryCount.get(countryItemTuple[0], [])
    curList.append(countryItemTuple[1])
    countryCount[countryItemTuple[0]] = curList
countryCount

{'Brazil': ['Sugar cane',
  'Sugar Crops Primary',
  'Cattle',
  'Sugar Raw Centrifugal',
  'Oranges',
  'Molasses',
  'Coffee, green',
  'Cashewapple',
  'Sisal',
  'Brazil nuts, with shell'],
 'China, mainland': ['Cereals, Total',
  'Vegetables Primary',
  'Eggs, hen, in shell',
  'Pigs',
  'Sheep and Goats',
  'Fruit Primary',
  'Rice, paddy',
  'Vegetables, fresh nes',
  'Sheep',
  'Roots and Tubers, Total',
  'Rice, paddy (rice milled equivalent)',
  'Wheat',
  'Potatoes',
  'Meat, Total',
  'Eggs, other bird, in shell',
  'Cucumbers and gherkins',
  'Watermelons',
  'Tomatoes',
  'Sweet potatoes',
  'Meat, pig',
  'Apples',
  'Citrus Fruit, Total',
  'Mushrooms and truffles',
  'Beer of barley',
  'Eggplants (aubergines)',
  'Cabbages and other brassicas',
  'Eggs Primary',
  'Spinach',
  'Onions, dry',
  'Garlic',
  'Seed cotton',
  'Tangerines, mandarins, clementines, satsumas',
  'Carrots and turnips',
  'Beans, green',
  'Groundnuts, with shell',
  'Chillies and peppers, gree

In [33]:
def getNumberOne(country):
    """
        Get the count and items a country 
        is the number one producer in and
        put it in a formatted string.
        Arguments:
            country: a country in the dataframe
        Returns: a String formatted like "[Country] is the number 
        one producer in # of items: item1, item2, ..."
    """
    itemList = countryCount.get(country, [])
    
    if (len(itemList) == 0):
        return "{Country} is not a top producer of any items".format(Country=country)
    else:
        countString = "From 2015 to 2020, {Country} is the top producer in {numItems} items".format(Country=country, 
                                                                                                    numItems=len(itemList))
        itemString = "which are: "
        if len(itemList) >= 10:
            itemString += "\n"
        counter = 0
        for item in itemList:
            if counter % 5 == 0:
                itemString += "\n"
            if counter == len(itemList) - 1:
                if len(itemList) > 1:
                    itemString += "and {item}.".format(item=item)
                else:
                    itemString += item + "."
            else:
                itemString += item + ", "
                counter += 1
        return countString + " " + itemString

In [34]:
getNumberOne("Canada")

'From 2015 to 2020, Canada is the top producer in 5 items which are: \nRapeseed, Peas, dry, Lentils, Canary seed, and Milk, skimmed evaporated.'

In [35]:
print(getNumberOne("China, mainland"))

From 2015 to 2020, China, mainland is the top producer in 97 items which are: 

Cereals, Total, Vegetables Primary, Eggs, hen, in shell, Pigs, Sheep and Goats, 
Fruit Primary, Rice, paddy, Vegetables, fresh nes, Sheep, Roots and Tubers, Total, 
Rice, paddy (rice milled equivalent), Wheat, Potatoes, Meat, Total, Eggs, other bird, in shell, 
Cucumbers and gherkins, Watermelons, Tomatoes, Sweet potatoes, Meat, pig, 
Apples, Citrus Fruit, Total, Mushrooms and truffles, Beer of barley, Eggplants (aubergines), 
Cabbages and other brassicas, Eggs Primary, Spinach, Onions, dry, Garlic, 
Seed cotton, Tangerines, mandarins, clementines, satsumas, Carrots and turnips, Beans, green, Groundnuts, with shell, 
Chillies and peppers, green, Pears, Lettuce and chicory, Peaches and nectarines, Grapes, 
Melons, other (inc.cantaloupes), Oil, soybean, Peas, green, Cauliflowers and broccoli, Pumpkins, squash and gourds, 
Asparagus, Plums and sloes, Fruit, citrus nes, Sheep and Goat Meat, Grapefruit (inc. pom